In [28]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import scipy
import altair as alt
from altair import datum
from tqdm.auto import tqdm, trange

from src.model import tscv
from src.model.metrics import corrected_rmse_score
from src.feature_engineering import add_lagged_features, drop_non_features, add_as_features, add_as_cat_features
from src.functional import comp, partial

%run constants.py

%matplotlib inline
print("Versions:")
print("  Python: %s" % sys.version)
for module in [pd, np, sns, sklearn, alt]:
    print("  %s: %s" %(module.__name__, module.__version__))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Versions:
  Python: 3.8.2 (default, Jul 16 2020, 14:00:26) 
[GCC 9.3.0]
  pandas: 1.1.0
  numpy: 1.19.1
  seaborn: 0.10.1
  sklearn: 0.23.2
  altair: 4.1.0


# Feature Engineering

Let's first load our train and test sets and set up our CV split.

In [29]:
import zipfile
with zipfile.ZipFile(os.path.join(RAW_DATA_DIR, 'competitive-data-science-predict-future-sales.zip'), 'r') as datasets_file:
    test_set = pd.read_csv(datasets_file.open('test.csv'))
train_set = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'train-set.parquet'))
test_subset = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'test-subset.parquet'))
sales_train_by_month = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'sales-train-by-month.parquet'))
cv_splits = tscv.split(train_set['date_block_num'].values)

Now let's start with the obvious one: the item counts. I wrote a function that calculates lagged features from a dataset.

Since we're dealing with lagged features, early months won't have any. Let's cut it down by half on both the window of lagged features and the train window for now.

In [3]:
cv_splits_w18 = tscv.split(train_set['date_block_num'].values, window=18)
train_set_w18 = add_lagged_features(train_set, sales_train_by_month, 'item_cnt', max_lag=18)
test_subset_w18 = add_lagged_features(test_subset, sales_train_by_month, 'item_cnt', max_lag=18)

In [4]:
train_set_w18.describe()

,date_block_num,item_id,shop_id,item_cnt_month,f__item_cnt_1,f__item_cnt_2,f__item_cnt_3,f__item_cnt_4,f__item_cnt_5,f__item_cnt_6,...,f__item_cnt_9,f__item_cnt_10,f__item_cnt_11,f__item_cnt_12,f__item_cnt_13,f__item_cnt_14,f__item_cnt_15,f__item_cnt_16,f__item_cnt_17,f__item_cnt_18
count,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,...,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06,1.609124e+06
mean,1.466479e+01,1.068099e+04,3.280585e+01,2.022806e+00,1.454636e+00,1.288606e+00,1.153339e+00,1.047351e+00,9.614697e-01,8.883274e-01,...,6.866053e-01,6.256970e-01,5.632481e-01,5.133849e-01,4.563769e-01,4.128818e-01,3.723100e-01,3.363905e-01,3.056452e-01,2.770955e-01
std,9.542322e+00,6.238883e+03,1.653701e+01,2.577964e+00,7.989895e+00,7.630533e+00,7.427049e+00,7.354710e+00,7.217674e+00,7.059026e+00,...,6.602638e+00,6.421906e+00,6.058962e+00,5.836906e+00,5.658653e+00,5.348509e+00,5.066624e+00,5.028181e+00,4.846775e+00,4.738297e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.200000e+01,-2.200000e+01,-4.000000e+00,-4.000000e+00,-2.000000e+00,-4.000000e+00,...,-4.000000e+00,-4.000000e+00,-4.000000e+00,-2.000000e+00,-4.000000e+00,-4.000000e+00,-4.000000e+00,-4.000000e+00,-2.000000e+00,-4.000000e+00
25%,6.000000e+00,5.045000e+03,2.100000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.400000e+01,1.049700e+04,3.100000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.300000e+01,1.606000e+04,4.700000e+01,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,3.300000e+01,2.216900e+04,5.900000e+01,2.000000e+01,1.305000e+03,1.305000e+03,1.305000e+03,1.305000e+03,1.305000e+03,1.305000e+03,...,1.305000e+03,1.305000e+03,1.305000e+03,1.305000e+03,1.305000e+03,1.305000e+03,9.890000e+02,1.305000e+03,1.305000e+03,1.305000e+03


In [5]:
test_subset_w18.describe()

,item_id,shop_id,date_block_num,f__item_cnt_1,f__item_cnt_2,f__item_cnt_3,f__item_cnt_4,f__item_cnt_5,f__item_cnt_6,f__item_cnt_7,...,f__item_cnt_9,f__item_cnt_10,f__item_cnt_11,f__item_cnt_12,f__item_cnt_13,f__item_cnt_14,f__item_cnt_15,f__item_cnt_16,f__item_cnt_17,f__item_cnt_18
count,28680.000000,28680.000000,28680.0,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000,...,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000,28680.000000
mean,10701.266213,31.995223,34.0,2.191388,1.330021,1.200558,1.001604,0.996130,1.155544,1.205370,...,0.938703,1.216423,1.809484,1.201430,0.936681,0.823989,0.843968,0.705753,0.733264,0.656032
std,6102.659012,16.488541,0.0,15.032652,6.592039,5.628176,5.637410,6.002198,10.252714,11.010553,...,4.748555,10.785793,14.693371,10.818708,7.937403,7.753421,7.355884,6.910303,8.234696,6.645365
min,30.000000,2.000000,34.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,5005.000000,19.000000,34.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10659.000000,31.000000,34.0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,15494.000000,46.000000,34.0,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,22167.000000,59.000000,34.0,2253.000000,473.000000,436.000000,482.000000,444.000000,742.000000,813.000000,...,257.000000,1000.000000,1209.000000,772.000000,634.000000,639.000000,591.000000,563.000000,771.000000,602.000000


In [6]:
X_train, y_train = drop_non_features(train_set_w18).values, train_set_w18['item_cnt_month'].values
X_test = drop_non_features(test_subset_w18).values

I'll use a default XGB regressor since in my experience it's usually a good default to start with.

In [7]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_validate
from src.model import ClippedOutputRegressor

reg = ClippedOutputRegressor(XGBRegressor(n_jobs=-1, verbosity=1))

In [8]:
scores = cross_validate(reg, X=X_train, y=y_train,
                        scoring=corrected_rmse_score, verbose=2, n_jobs=-1, 
                        cv=cv_splits_w18, return_train_score=True)
scores

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  3.9min finished


{'fit_time': array([222.67374158, 218.43320179, 213.79310656]),
 'score_time': array([0.29977012, 0.25102425, 0.26123786]),
 'test_score': array([-0.77165437, -1.02308756, -1.03266588]),
 'train_score': array([-0.92676242, -0.92933981, -0.93133453])}

In [9]:
scores['test_score'].mean(), scores['test_score'].std() 

(-0.9424692693294418, 0.12084765474984473)

Pretty close to our previous dummy test, so not good. Let's try adding some categorical features: the previous test had the date block num, so we'll go even further and add the date ids.

In [10]:
date_ids = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'date-ids.parquet'))
date_ids.head()

,date_block_num,month_id,year_id
0,0,0,0
1,1,1,0
2,2,2,0
3,3,3,0
4,4,4,0


In [11]:
train_set_w18_date_ids = add_as_cat_features(train_set_w18.merge(date_ids, on='date_block_num', how='left', sort=False), 
                                             ['date_block_num', 'month_id', 'year_id'])
X_train = drop_non_features(train_set_w18_date_ids).values

In [12]:
scores = cross_validate(reg, X=X_train, y=y_train,
                        scoring=corrected_rmse_score, verbose=2, n_jobs=-1, 
                        cv=cv_splits_w18, return_train_score=True)
scores

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  4.2min finished


{'fit_time': array([244.08122063, 239.6075933 , 233.86898255]),
 'score_time': array([0.3487978 , 0.30139661, 0.31716919]),
 'test_score': array([-0.7724726 , -1.03043161, -1.0483989 ]),
 'train_score': array([-0.91004636, -0.91209181, -0.91578144])}

In [13]:
scores['test_score'].mean(), scores['test_score'].std() 

(-0.9504343724564208, 0.12605157671435444)

Well, it's worse. Let's add the base ids to the train set and run it again.

In [14]:
train_set_w18_date_ids_base_ids = add_as_cat_features(train_set_w18_date_ids, ['item_id', 'shop_id'])

In [15]:
X_train = drop_non_features(train_set_w18_date_ids_base_ids).values

scores = cross_validate(reg, X=X_train, y=y_train,
                        scoring=corrected_rmse_score, verbose=2, n_jobs=-1, 
                        cv=cv_splits, return_train_score=True)
scores

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:  8.7min finished


{'fit_time': array([469.65439868, 472.22846842, 493.10321259]),
 'score_time': array([0.55858803, 0.49258566, 0.51222968]),
 'test_score': array([-0.7694865 , -0.98612991, -1.06177736]),
 'train_score': array([-0.8874763 , -0.87917883, -0.88494101])}

In [16]:
scores['test_score'].mean(), scores['test_score'].std() 

(-0.9391312550717849, 0.12386859351220451)

A little better. Let's try the whole dataset to see if that's any better

In [34]:
def add_date_ids(df):
    return add_as_cat_features(df.merge(date_ids, on='date_block_num', how='left', sort=False), 
                                        ['date_block_num', 'month_id', 'year_id'])

transform = comp(
    lambda df: add_as_cat_features(df, ['item_id', 'shop_id']),
    add_date_ids,
    lambda df: add_lagged_features(df, sales_train_by_month, 'item_cnt'))

train_set_lagged_date_ids_base_ids = transform(train_set)

In [36]:
X_train = drop_non_features(train_set_lagged_date_ids_base_ids).values
scores = cross_validate(reg, X=X_train, y=y_train,
                        scoring=corrected_rmse_score, verbose=2, n_jobs=-1, 
                        cv=cv_splits, return_train_score=True)
scores

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed: 12.0min finished


{'fit_time': array([669.21315479, 669.66440964, 693.05559611]),
 'score_time': array([0.57962251, 0.50677729, 0.52487874]),
 'test_score': array([-0.76700048, -0.98716346, -1.03798434]),
 'train_score': array([-0.88615257, -0.88542616, -0.89052309])}

In [37]:
scores['test_score'].mean(), scores['test_score'].std() 

(-0.9307160921631675, 0.11760893479275158)

OK, we can see it's an improvement. Let's just submit that and see how it does on the public LB.

In [30]:
test_subset_lagged_date_ids_base_ids = transform(test_subset)
X_test = drop_non_features(test_subset_lagged_date_ids_base_ids).values

In [38]:
from src.submission import submission_from_subset

reg.fit(X_train, y_train)
test_subset_lagged_date_ids_base_ids['item_cnt_month'] = reg.predict(X_test)

submission = submission_from_subset(test_subset_lagged_date_ids_base_ids, test_set)
submission.to_csv(os.path.join(TMP_DIR, 'xgb-dataset-01.csv'), index=False)

In [39]:
%%bash
kaggle c submit -f ${TMP_DIR}/xgb-dataset-01.csv -m 'Default XGB with date ids and lagged item counts' competitive-data-science-predict-future-sales

Successfully submitted to Predict Future Sales

100%|██████████| 2.30M/2.30M [00:40<00:00, 59.3kB/s]


1.08752, not bad considering we didn't run any hyperparameter optimization.